## Import Library

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


## Dataset

In [15]:
df = pd.read_csv("vgsales.csv")

df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


## Cleaning Dataset

In [16]:
impute_mean = SimpleImputer(strategy='mean')
impute_modus = SimpleImputer(strategy='most_frequent' )

In [17]:
df['Year'] = impute_mean.fit_transform(df[['Year']])
df['Publisher'] = impute_modus.fit_transform(df[['Publisher']])

In [18]:
df.isna().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

## Data Spliting

Mengganti Value pada Tabel Genre

In [19]:
df.replace("Action", value=0, inplace=True)
df.replace("Sports", value=1, inplace=True)
df.replace("Misc", value=2, inplace=True)
df.replace("Role-Playing", value=3, inplace=True)
df.replace("Shooter", value=4, inplace=True)
df.replace("Adventure", value=5, inplace=True)
df.replace("Racing", value=6, inplace=True)
df.replace("Platform", value=7, inplace=True)
df.replace("Simulation", value=8, inplace=True)
df.replace("Fighting", value=9, inplace=True)
df.replace("Strategy", value=10, inplace=True)
df.replace("Puzzle", value=11, inplace=True)

In [20]:
X = df[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]]
y = df[["Genre"]]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.3)

#### Encoding label

In [22]:
num_class = 12
y_train = tf.keras.utils.to_categorical(y_train, num_class)
y_test = tf.keras.utils.to_categorical(y_test, num_class)

### Sequential Model

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu', input_dim=5),
    tf.keras.layers.Dense(num_class, activation='softmax'),
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 12)                72        
                                                                 
 dense_3 (Dense)             (None, 12)                156       
                                                                 
Total params: 228
Trainable params: 228
Non-trainable params: 0
_________________________________________________________________


### Compile Model

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

### Training Model

In [25]:
history = model.fit(
    x = X_train, y = y_train, 
    validation_data=(X_test, y_test),
    verbose=1,
    epochs=50,
    steps_per_epoch=50
)

Epoch 1/50
50/50 [==============================] - 1s 8ms/step - loss: 2.4950 - accuracy: 0.1591 - val_loss: 2.4731 - val_accuracy: 0.1841
Epoch 2/50
50/50 [==============================] - 0s 5ms/step - loss: 2.4522 - accuracy: 0.2006 - val_loss: 2.4409 - val_accuracy: 0.1980
Epoch 3/50
50/50 [==============================] - 0s 5ms/step - loss: 2.4244 - accuracy: 0.2061 - val_loss: 2.4165 - val_accuracy: 0.1990
Epoch 4/50
50/50 [==============================] - 0s 5ms/step - loss: 2.4028 - accuracy: 0.2046 - val_loss: 2.3978 - val_accuracy: 0.2000
Epoch 5/50
50/50 [==============================] - 0s 5ms/step - loss: 2.3855 - accuracy: 0.2034 - val_loss: 2.3825 - val_accuracy: 0.1994
Epoch 6/50
50/50 [==============================] - 0s 5ms/step - loss: 2.3720 - accuracy: 0.2030 - val_loss: 2.3710 - val_accuracy: 0.2002
Epoch 7/50
50/50 [==============================] - 0s 5ms/step - loss: 2.3618 - accuracy: 0.2030 - val_loss: 2.3623 - val_accuracy: 0.2006
Epoch 8/50
50/50 [==

Akurasi Data

In [26]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Loss   :', score[0])
print('Akurasi:', score[1])

Loss   : 2.303873300552368
Akurasi: 0.21124498546123505
